# Fractal ML Training

## Setup

In [4]:
import time
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
from sparkmeasure import StageMetrics

In [ ]:
data_path = "/opt/spark/work-dir/data/FRACTAL"
sample_fraction = 2
num_executors = 2

In [ ]:
spark = SparkSession.builder.appName("fractal-cv-rf") \
    .master("spark://spark-master:7077") \
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .config("spark.hadoop.fs.s3a.aws.credentials.provider", "com.amazonaws.auth.DefaultAWSCredentialsProviderChain") \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "2g") \
    .config("spark.executor.cores", "2") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "1") \
    .config("spark.dynamicAllocation.maxExecutors", str(num_executors)) \
    .config("spark.dynamicAllocation.initialExecutors", str(num_executors)) \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "/opt/spark/spark-events") \
    .getOrCreate()

In [7]:
stage_metrics = StageMetrics(spark)
print(f"Spark UI: {spark.sparkContext.uiWebUrl}")

Spark UI: http://59d281bd1cfe:4040


## Data Loading

In [8]:
def prepare_data(df):
    df = df.withColumn("z_raw", col("xyz")[2]).withColumn(
        "ndvi",
        when(
            (col("Infrared") + col("Red")) != 0,
            (col("Infrared") - col("Red")) / (col("Infrared") + col("Red")),
        ).otherwise(0),
    )
    return df.select("z_raw", "Intensity", "Red", "Green", "Blue", "Infrared", "ndvi", col("Classification").alias("label"))


def load_sample(spark, path, fraction, cols):
    import random
    
    sc = spark.sparkContext
    hadoop_conf = sc._jsc.hadoopConfiguration()
    fs = spark._jvm.org.apache.hadoop.fs.FileSystem.get(hadoop_conf)
    path_obj = spark._jvm.org.apache.hadoop.fs.Path(path)
    
    file_list = []
    if fs.exists(path_obj):
        statuses = fs.listStatus(path_obj)
        for status in statuses:
            file_path = str(status.getPath())
            if file_path.endswith('.parquet') or not '.' in file_path.split('/')[-1]:
                file_list.append(file_path)
    
    if not file_list:
        file_list = [path]
    
    random.seed(62)
    
    if fraction <= 1.0:
        num_files = max(1, int(len(file_list) * fraction))
        print(f"Sampling {num_files}/{len(file_list)} files (fraction={fraction})")
    else:
        num_files = min(int(fraction), len(file_list))
        print(f"Sampling {num_files}/{len(file_list)} files (count={int(fraction)})")
    
    sampled_files = random.sample(file_list, num_files)
    print(sampled_files)
    df = spark.read.parquet(*sampled_files).select(*cols)
    
    df = prepare_data(df).cache()
    print(f"Loaded {df.count()} rows")
    return df

In [ ]:
cols = ["xyz", "Intensity", "Classification", "Red", "Green", "Blue", "Infrared"]

stage_metrics.begin()
start_time = time.time()

train = load_sample(spark, f"{data_path}/train/", sample_fraction, cols)
val = load_sample(spark, f"{data_path}/val/", sample_fraction, cols)
test = load_sample(spark, f"{data_path}/test/", sample_fraction, cols)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

[Stage 2:>                                                      (53 + 4) / 4602]

## Pipeline

In [ ]:
z_assembler = VectorAssembler(inputCols=["z_raw"], outputCol="z_vec")
z_scaler = StandardScaler(inputCol="z_vec", outputCol="z", withMean=False, withStd=True)

assembler = VectorAssembler(
    inputCols=["z", "Intensity", "Red", "Green", "Blue", "Infrared", "ndvi"],
    outputCol="features",
)

rf = RandomForestClassifier(labelCol="label", featuresCol="features", seed=62)
pipeline = Pipeline(stages=[z_assembler, z_scaler, assembler, rf])

## CrossValidator

In [17]:
paramGrid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [50, 100, 200]) \
    .addGrid(rf.maxDepth, [10, 15, 20]) \
    .build()

evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)

n_folds = 1

cv = CrossValidator(
    estimator=pipeline,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=n_folds,
    parallelism=num_executors,
    seed=62
)

print(f"Training {len(paramGrid)} models × {n_folds} folds = {len(paramGrid) * n_folds} fits")

Training 9 models × 1 folds = 9 fits


In [18]:
cv_model = cv.fit(train)

25/11/09 20:54:44 WARN CacheManager: Asked to cache already cached data.
25/11/09 20:54:44 WARN CacheManager: Asked to cache already cached data.
25/11/09 20:54:44 WARN TaskSetManager: Lost task 0.0 in stage 59.0 (TID 114) (172.19.0.6 executor 0): java.lang.IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.stat.SummarizerBuffer.mean(Summarizer.scala:624)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.$anonfun$eval$1(Summarizer.scala:358)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.sc

Py4JJavaError: An error occurred while calling o402074.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 59.0 failed 4 times, most recent failure: Lost task 0.3 in stage 59.0 (TID 120) (172.19.0.5 executor 1): java.lang.IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.stat.SummarizerBuffer.mean(Summarizer.scala:624)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.$anonfun$eval$1(Summarizer.scala:358)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.

## Results

In [ ]:
best_model = cv_model.bestModel
best_params = {
    "numTrees": best_model.stages[-1].getNumTrees,
    "maxDepth": best_model.stages[-1].getMaxDepth(),
}

val_accuracy = evaluator.evaluate(cv_model.transform(val))
test_accuracy = evaluator.evaluate(cv_model.transform(test))

print(f"Best: {best_params}")
print(f"Val: {val_accuracy:.4f}, Test: {test_accuracy:.4f}")

25/11/09 20:54:44 WARN TaskSetManager: Lost task 0.0 in stage 65.0 (TID 128) (172.19.0.6 executor 0): java.lang.IllegalArgumentException: requirement failed: Nothing has been added to this summarizer.
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.ml.stat.SummarizerBuffer.mean(Summarizer.scala:624)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.$anonfun$eval$1(Summarizer.scala:358)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:286)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62)
	at scala.collection.mutable.ResizableArray.foreach$(ResizableArray.scala:55)
	at scala.collection.mutable.ArrayBuffer.foreach(ArrayBuffer.scala:49)
	at scala.collection.TraversableLike.map(TraversableLike.scala:286)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:279)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.ml.stat.SummaryBuilderImpl$MetricsAggregate.eval

## Metrics

In [ ]:
stage_metrics.end()
metrics = stage_metrics.aggregate_stage_metrics()
total_time = (metrics['elapsedTime'] / 1000.0) if metrics else 0

print(f"Time: {total_time:.2f}s")
print(f"CPU: {metrics.get('executorCpuTime', 0) / 1000.0:.2f}s")
print(f"Shuffle Read: {metrics.get('shuffleReadBytes', 0) / (1024**3):.2f} GB")
print(f"Shuffle Write: {metrics.get('shuffleWriteBytes', 0) / (1024**3):.2f} GB")

## Cleanup

In [ ]:
train.unpersist()
val.unpersist()
test.unpersist()
spark.stop()